In [ ]:
# Imports
import utils
import os
import pickle
import pandas as pd
import numpy as np
import tensorflow as tf
import cv2
import concurrent.futures
import joblib
from tabulate import tabulate

from sklearn.metrics import accuracy_score, balanced_accuracy_score, mean_squared_error, precision_score, recall_score, r2_score

Define helper functions

In [ ]:
CLASSIC_DESCRIPTORS_EXTENSION = ".joblib"

In [ ]:
def evaluate_predictions(model_name_list, predictions_list, targets, is_classification, is_binary_classification, n_classes):
    # Checks
    if not is_classification:
        assert n_classes is None
    else:
        if is_binary_classification:
            assert n_classes == 2
        else:
            assert n_classes > 2
    if is_classification:
        headers = ["model_name", "Acc", "Balanced_accuracy", "Precision", "Recall"]
    else:
        headers = ["MSE", "R²"]
        
    rows = []
    for model_name, predictions in zip(model_name_list, predictions_list):
        if is_classification:
            accuracy = accuracy_score(targets, predictions)
            balanced_accuracy = balanced_accuracy_score(targets, predictions)
            if is_binary_classification:
                precision = precision_score(targets, predictions)
                recall = recall_score(targets, predictions)
            else:
                precision = precision_score(targets, predictions, average='macro')
                recall = recall_score(targets, predictions, average='macro')
            rows.append([model_name, accuracy, balanced_accuracy, precision, recall])
        else:
            mse = mean_squared_error(targets, predictions)
            r2 = r2_score(targets, predictions)
            rows.append([model_name, mse, r2])
    formatted_rows = [[row[0]]+[f"{score:.3f}" for score in row[1:]] for row in rows]
    
    print(tabulate(formatted_rows, headers=headers))

In [ ]:
def evaluate_classic_descriptor_predictions(model_name_list, predictions_list, targets, is_classification, is_binary_classification, n_classes):
    evaluate_predictions(model_name_list, predictions_list, targets, is_classification, is_binary_classification, n_classes)

In [ ]:
def get_partial_load_image(image_method_folder):
    def load_image(img_path):
        return cv2.imread(
            os.path.join(image_method_folder, img_path)
        )
    
    return load_image

In [ ]:
def evaluate_classic_descriptors(dataset_name):
    # Get the path for the models
    models_path = utils.get_classicdescriptors_path(dataset_name)

    models_to_evaluate = [
        m for m in os.listdir(models_path) if m.endswith(CLASSIC_DESCRIPTORS_EXTENSION)
    ]

    if not models_to_evaluate:
        return
    
    # Load the data
    X,y = utils.get_X_y(dataset_name)

    # Select the test data
    indices = utils.get_indices_test(dataset_name)
    X_test, y_test = X[indices], y[indices]
    del X
    del y

    is_classification = utils.is_dataset_classification(dataset_name)
    is_binary_classification = utils.is_dataset_binary_classification(dataset_name)
    n_classes = utils.get_number_of_classes(dataset_name) if is_classification else None

    model_name_list, predictions_list = [], []

    # Iterate over all the models
    for model_name in models_to_evaluate:
        model = joblib.load(os.path.join(models_path, model_name))
        model_name_list.append(model_name)

        # Calculate the predictions
        predictions_list.append(model.predict(X_test))

    # Evaluate the predictions
    evaluate_classic_descriptor_predictions(
        model_name_list = model_name_list, 
        predictions_list = predictions_list,
        targets = y_test,
        is_classification = is_classification,
        is_binary_classification = is_binary_classification,
        n_classes = n_classes
    )

In [ ]:
def evaluate_cnn_predictions(model_name_list, predictions_list, targets, is_classification, is_binary_classification, n_classes):
    predictions_list = [predictions.flatten() for predictions in predictions_list]
    if is_classification:
        if is_binary_classification:
            predictions_list = [(predictions > 0.5).astype(int) for predictions in predictions_list]
        else:
            print(predictions_list[0].shape, targets.shape)
            raise NotImplementedError()
    else:
        predictions_list = predictions_list
    evaluate_predictions(model_name_list, predictions_list, targets, is_classification, is_binary_classification, n_classes)

In [ ]:
def evaluate_cnn_models(dataset_name):
    model_name_list, predictions_list = [], []

    # Get path to models
    model_paths = utils.get_cnnmodels_path(dataset_name)

    all_model_names = [f for f in os.listdir(model_paths)]
    if not all_model_names:
        return
    
    # Get X and y indices
    X,y = utils.get_X_y(dataset_name)
    indices_test = utils.get_indices_test(dataset_name)
    y_test = y[indices_test]
    del X,y

    for image_method in utils.ALL_IMAGE_METHODS:
        # List all the models
        model_names = [f for f in all_model_names if f.startswith(image_method)]

        # Get path to images
        images_path = utils.get_images_path_for_dataset(dataset_name, image_method)

        is_classification = utils.is_dataset_classification(dataset_name)
        is_binary_classification = utils.is_dataset_binary_classification(dataset_name)
        n_classes = utils.get_number_of_classes(dataset_name) if is_classification else None

        # Load the images
        csv_name = next(f for f in os.listdir(images_path) if f.endswith(".csv"))
        csv_file = os.path.join(images_path, csv_name)
        images_paths = pd.read_csv(csv_file)["images"].to_numpy()[indices_test]
        func_load_image = get_partial_load_image(images_path)
        with concurrent.futures.ThreadPoolExecutor() as executor:
            X_test_img = np.array(list(executor.map(func_load_image, images_paths)))

        # Iterate over the methods
        for model_name in sorted(model_names):
            # Load the model
            model_path = os.path.join(model_paths, model_name)
            model = tf.keras.models.load_model(model_path)

            # Get the predictions
            predictions = model.predict(X_test_img, verbose=0)

            model_name_list.append(model_name)
            predictions_list.append(predictions)

    # Calculate the metrics
    evaluate_cnn_predictions(
        model_name_list = model_name_list, 
        predictions_list = predictions_list,
        targets = y_test,
        is_classification=is_classification,
        is_binary_classification=is_binary_classification,
        n_classes=n_classes
    )

In [ ]:
def evaluate_cnn_mlp_models(dataset_name):
    model_name_list, predictions_list = [], []

    # Get path to models
    model_paths = utils.get_cnnmlp_models_path(dataset_name)

    all_model_names = [f for f in os.listdir(model_paths)]
    if not all_model_names:
        return
    
    # Get X and y indices
    X,y = utils.get_X_y(dataset_name)
    indices_test = utils.get_indices_test(dataset_name)
    X_test_data = X[indices_test]
    y_test = y[indices_test]
    del X,y

    for image_method in utils.ALL_IMAGE_METHODS:

        # List all the models
        model_names = [f for f in all_model_names if f.startswith(image_method)]

        if not model_names:
            continue

        # Get path to images
        images_path = utils.get_images_path_for_dataset(dataset_name, image_method)

        is_classification = utils.is_dataset_classification(dataset_name)
        is_binary_classification = utils.is_dataset_binary_classification(dataset_name)
        n_classes = utils.get_number_of_classes(dataset_name) if is_classification else None

        # Load the images
        csv_name = next(f for f in os.listdir(images_path) if f.endswith(".csv"))
        csv_file = os.path.join(images_path, csv_name)
        images_paths = pd.read_csv(csv_file)["images"].to_numpy()[indices_test]
        func_load_image = get_partial_load_image(images_path)
        with concurrent.futures.ThreadPoolExecutor() as executor:
            X_test_img = np.array(list(executor.map(func_load_image, images_paths)))

        # Iterate over the methods
        for model_name in sorted(model_names):
            # Load the model
            model_path = os.path.join(model_paths, model_name)
            model = tf.keras.models.load_model(model_path)

            # Get the predictions
            predictions = model.predict([X_test_img, X_test_data], verbose=0)

            model_name_list.append(model_name)
            predictions_list.append(predictions)

    # Calculate the metrics
    evaluate_cnn_predictions(
        model_name_list = model_name_list, 
        predictions_list = predictions_list,
        targets = y_test,
        is_classification=is_classification,
        is_binary_classification=is_binary_classification,
        n_classes=n_classes
    )

In [ ]:
def evaluate_cnn_ml_models(dataset_name):
    model_name_list, predictions_list = [], []

    # Get path to models
    model_paths = utils.get_cnn_classicdescriptors_path(dataset_name)

    all_model_names = [f for f in os.listdir(model_paths)]
    if not all_model_names:
        return
    
    # Get X and y indices
    X,y = utils.get_X_y(dataset_name)
    indices_test = utils.get_indices_test(dataset_name)
    X_test_data = X[indices_test]
    y_test = y[indices_test]
    del X,y
    
    for image_method in utils.ALL_IMAGE_METHODS:
        # List all the models
        model_names = [f for f in all_model_names if f.startswith(image_method)]

        if not model_names:
            continue

        # Get path to images
        images_path = utils.get_images_path_for_dataset(dataset_name, image_method)

        is_classification = utils.is_dataset_classification(dataset_name)
        is_binary_classification = utils.is_dataset_binary_classification(dataset_name)
        n_classes = utils.get_number_of_classes(dataset_name) if is_classification else None

        # Load the images
        csv_name = next(f for f in os.listdir(images_path) if f.endswith(".csv"))
        csv_file = os.path.join(images_path, csv_name)
        images_paths = pd.read_csv(csv_file)["images"].to_numpy()[indices_test]
        func_load_image = get_partial_load_image(images_path)
        with concurrent.futures.ThreadPoolExecutor() as executor:
            X_test_img = np.array(list(executor.map(func_load_image, images_paths)))

        # Iterate over the methods
        for model_name in sorted(model_names):
            # Load the model
            model_path = os.path.join(model_paths, model_name)
            model = tf.keras.models.load_model(model_path)

            classic_model = joblib.load(
                os.path.join(
                    utils.get_classicdescriptors_split1_path(dataset_name),
                    model_name.split("_")[1].strip()
                )
            )
            X_classic_predictions = classic_model.predict(X_test_data)

            # Get the predictions
            predictions = model.predict([X_test_img, X_classic_predictions], verbose=0)

            model_name_list.append(model_name)
            predictions_list.append(predictions)
    
    # Calculate the metrics
    evaluate_cnn_predictions(
        model_name_list = model_name_list, 
        predictions_list = predictions_list,
        targets = y_test,
        is_classification=is_classification,
        is_binary_classification=is_binary_classification,
        n_classes=n_classes
    )

Run evaluations

In [ ]:
evaluate_classic_descriptors(dataset_name=utils.HELOC_NAME)

In [ ]:
evaluate_cnn_models(dataset_name=utils.HELOC_NAME)

In [ ]:
evaluate_cnn_mlp_models(dataset_name=utils.HELOC_NAME)

In [ ]:
evaluate_cnn_ml_models(dataset_name=utils.HELOC_NAME)